In [1]:
import json
import pandas as pd

match_id = "EFBO2372905"

with open("./matches/EFBO2372905.json") as f:
    data = f.read()

data = json.loads(data)
match_key = next(iter(data["matchData"][0]["tournamentDatesWithEvents"]))
game_date = data["matchData"][0]["tournamentDatesWithEvents"][match_key][0]["events"][0]["startTime"][:10]


with open(f"./line-ups/{match_id}.json") as f:
    data = f.read()

data = json.loads(data)
home_team = data[0]["data"]["teams"]["homeTeam"]["name"]
away_team = data[0]["data"]["teams"]["awayTeam"]["name"]

if home_team == "Tranmere Rovers":
    team = 'homeTeam'
else:
    team = 'awayTeam'

formation = data[0]["data"]["teams"][team]["formation"]
formation = "-".join(str(formation))

players = data[0]["data"]["teams"][team]["players"]

apps_df = pd.DataFrame()
cards_df = pd.DataFrame()
subs_df = pd.DataFrame()
sub_mins_df = pd.DataFrame()

for player in players:
    player_name = player["name"]["full"]
    shirt_no = player["meta"]["uniformNumber"]
    role = player["meta"]["status"].replace("bench", "sub")
    
    player_app = {
        "game_date": game_date,
        "player_name": player_name,
        "shirt_no": shirt_no,
        "role": role
    }
    player_app = pd.DataFrame(player_app, index = [0])

    apps_df = pd.concat([apps_df, player_app], axis = 0)

    cards = player["bookings"]
    if cards:
        player_cards = pd.json_normalize(player["bookings"])
        player_cards["game_date"] = game_date
        player_cards["player_name"] = player_name
        cards_df = pd.concat([cards_df, player_cards], axis = 0)

    subs = player["substitutions"]
    if subs:
        sub_min = subs[0]["timeElapsed"]
        sub_on_no = subs[0]["replacedBy"]["meta"]["uniformNumber"]
        player_on = subs[0]["replacedBy"]["name"]["full"]
        sub_mins_pld = subs[0]["timeElapsed"]
        sub_mins = [{
            "game_date": game_date,
            "player_name": player_name,
            "min_off": sub_min,
            "min_on": ""
        },
        {
            "game_date": game_date,
            "player_name": player_on,
            "min_off": "",
            "min_on": sub_min
        }]
        sub_mins = pd.DataFrame(sub_mins)
        sub_mins_df = pd.concat([sub_mins_df, sub_mins], axis = 0)

        sub_names = [{
            "game_date": game_date,
            "shirt_no": sub_on_no,
            "player_name": player_name,
            "on_for": shirt_no,
            "off_for": ""
        },
        {
            "game_date": game_date,
            "shirt_no": shirt_no,
            "player_name": player_on,
            "on_for": "",
            "off_for": sub_on_no
        }]
        sub_names = pd.DataFrame(sub_names)
        subs_df = pd.concat([subs_df, sub_names], axis = 0)
    
yellows_df = cards_df.query("type == 'yellow-card'")
reds_df = cards_df.query("type.str.contains('red')")

if not yellows_df.empty:
    yellows_df = yellows_df[["game_date", "player_name"]]
    yellows_df["yellow_card"] = 1
if not reds_df.empty:
    reds_df = reds_df[["game_date", "player_name", "timeElapsed"]].rename(columns = {"timeElapsed": "min_so"})

apps_df.to_csv("./updates/updates-apps.csv", index = False)
subs_df.to_csv("./updates/updates-subs.csv", index = False)
sub_mins_df.to_csv("./updates/updates-sub-mins.csv", index = False)
yellows_df.to_csv("./updates/updates-yellows.csv", index = False)
reds_df.to_csv("./updates/updates-reds.csv", index = False)

In [10]:
attendance = data[0]["data"]["meta"]["attendance"].replace(",", "")
attendance = int(attendance)
attendance

4343